Θεόδωρος Αϊβαλής, Α.Μ.: 03117099

Θεοδότη Στόικου, Α.Μ.: 03117085


#Imports

In [ ]:
!pip install opencv-python==3.4.2.17 opencv-contrib-python==3.4.2.17

import cv2
import numpy as np
import sys
import scipy
from scipy import signal
from scipy.ndimage import laplace
import matplotlib.pyplot as plt
from numpy import linalg as LA
import math
from matplotlib.patches import Circle
from math import ceil, floor

print(cv2.__version__)

%matplotlib inline

#Μέρος 1: Ανίχνευση Ακμών σε Γκρίζες Εικόνες

##1.1. Δημιουργία Εικόνων Εισόδου

###1.1.1

In [ ]:
img = cv2.imread('edgetest_10.png', cv2.IMREAD_GRAYSCALE)
print("Resolution: ", img.shape)
print("Range: %d - %d " % (img.min(), img.max()))
img = img.astype(np.float)/255
plt.imshow(img, cmap='gray')

###1.1.2

####i) PSNR=20 dB

In [ ]:
mu, sigma1 = 0, 0.1 
s1 = np.random.normal(mu, sigma1, (512, 512))
s1= s1.reshape(512, 512)
noisy1 = img + s1
plt.imshow(noisy1, cmap='gray')

####ii) PSNR=10 dB

In [ ]:
mu, sigma2 = 0, 0.316
s2 = np.random.normal(mu, sigma2, (512, 512))
s2= s2.reshape(512, 512)
noisy2 = img + s2
plt.imshow(noisy2, cmap='gray')

##1.2. Υλοποίηση Αλγορίθμων Ανίχνευσης Ακμών

###1.2.1

####i) Διδιάστατη Gaussian Gσ(x, y)

In [ ]:
def gaussianfil(image, sigma):
  n = int(2*np.ceil(3*sigma)+1)
  gauss1D = cv2.getGaussianKernel(n, sigma) # Column vector
  gauss2D = gauss1D @ gauss1D.T # Symmetric gaussian kernel
  img_smooth = cv2.filter2D(image, -1, gauss2D)
  print(gauss1D.shape, gauss1D.T.shape, gauss2D.shape)
  plt.figure()
  plt.imshow(img_smooth, cmap='gray')
  plt.title("2D Gaussian")
  return img_smooth

In [ ]:
res = gaussianfil(noisy1, 2)

####ii) Laplacian-of-Gaussian (LoG) h(x, y) = ∇2Gσ(x, y).

In [ ]:
def logfil(image, sigma):
  n = int(2*np.ceil(3*sigma)+1)
  ax = np.linspace(-(n - 1) / 2., (n - 1) / 2., n)
  xx, yy = np.meshgrid(ax, ax)
  kernel_log = (-1)/(np.pi*sigma**4)*(1-(xx**2+yy**2)/(2*sigma**2))*np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sigma))
  img_smooth = cv2.filter2D(image, -1, kernel_log)
  plt.figure()
  plt.imshow(img_smooth, cmap='gray')
  plt.title("Laplacian-of-Gaussian") 
  return img_smooth

In [ ]:
res = logfil(noisy1, 2)

###1.2.2

####i) Γραμμική (L1)

In [ ]:
def linearL(image, sigma):
  n = int(2*np.ceil(3*sigma)+1)
  gauss1D = cv2.getGaussianKernel(n, sigma) # Column vector
  gauss2D = gauss1D @ gauss1D.T # Symmetric gaussian kernel
  L1= cv2.Laplacian(scipy.signal.convolve(gauss2D, image), cv2.CV_64F)
  plt.figure()  
  plt.imshow(L1, cmap='gray')
  plt.title("Linear_Laplacian")
  return L1

In [ ]:
res = linearL(noisy1, 2)

####ii) Μη-γραμμική (L2)

In [ ]:
def nonlinearL(image, sigma):
  B = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
  n = int(2*np.ceil(3*sigma)+1)
  gauss1D = cv2.getGaussianKernel(n, sigma) # Column vector
  gauss2D = gauss1D @ gauss1D.T # Symmetric gaussian kernel
  Is = scipy.signal.convolve(gauss2D, image)
  L2 = cv2.dilate(Is,B,iterations = 1) + cv2.erode(Is,B,iterations = 1) - 2*Is
  plt.figure()
  plt.imshow(L2, cmap = 'gray')
  plt.title("Non_Linear_Laplacian")
  return L2

In [ ]:
res = nonlinearL(noisy1, 2)

###1.2.3

In [ ]:
def zerocrossingsL(image, sigma):
  B = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
  L = logfil(image, sigma)
  ret, X = cv2.threshold(L, 0, 1, cv2.THRESH_BINARY)
  Y = cv2.dilate(X,B,iterations = 1) - cv2.erode(X,B,iterations = 1)
  plt.figure()
  plt.imshow(Y, cmap = 'gray_r')
  plt.title("zeroCrossings")
  return Y

In [ ]:
res = zerocrossingsL(noisy1, 2)

###1.2.4

In [ ]:
def select_zcL(image, sigma, th):
  Y = zerocrossingsL(image, sigma)
  n = int(2*np.ceil(3*sigma)+1)
  gauss1D = cv2.getGaussianKernel(n, sigma) # Column vector
  gauss2D = gauss1D @ gauss1D.T # Symmetric gaussian kernel
  Is = scipy.signal.convolve(gauss2D, image)
  gradimage = np.gradient(Is)
  rows, columns = gradimage[0].shape
  gradarray = np.zeros((rows, columns))
  for i in range (rows):
    for j in range (columns):
      gradarray[i][j] = np.sqrt(gradimage[0][i][j]**2 + gradimage[1][i][j]**2)
  limit = th * np.amax(gradarray)
  (rowsY, columnsY) = Y.shape
  for i in range (rowsY):
    for j in range (columnsY):
      if (Y[i][j] == 1):
        if ((gradarray[i][j]) <= limit):
          Y[i][j] = 0
  plt.figure()  
  plt.imshow(Y, cmap = 'gray_r')
  plt.title("select_zeroCrossings")
  return Y  

In [ ]:
res =select_zcL(noisy1, 1, 0.2)

In [ ]:
def EdgeDetect(image, sigma, th, laplacian):
  if (laplacian == 'linear'):
    Is = linearL(image, sigma)
  elif (laplacian == 'nonlinear'):
    Is = nonlinearL(image, sigma)
  Y = zerocrossingsL(image, sigma)
  gradimage = np.gradient(Is)
  rows, columns = gradimage[0].shape
  gradarray = np.zeros((rows, columns))
  for i in range (rows):
    for j in range (columns):
      gradarray[i][j] = np.sqrt(gradimage[0][i][j]**2 + gradimage[1][i][j]**2)
  limit = th * np.amax(gradarray)
  (rowsY, columnsY) = Y.shape
  for i in range (rowsY):
    for j in range (columnsY):
      if (Y[i][j] == 1):
        if ((gradarray[i][j]) <= limit):
          Y[i][j] = 0
  D = Y
  plt.figure()
  plt.imshow(D, cmap = 'gray_r')
  plt.title("EdgeDetect")
  return D  

In [ ]:
res = EdgeDetect(noisy1, 1, 0.2, 'nonlinear')

##1.3. Αξιολόγηση των Αποτελεσμάτων Ανίχνευσης Ακμών

###1.3.1

In [ ]:
def real_edges(image, th_real):
  B = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
  M = cv2.dilate(image,B,iterations = 1) - cv2.erode(image,B,iterations = 1)
  ret, T = cv2.threshold(M, th_real, 1, cv2.THRESH_BINARY)
  plt.figure()
  plt.imshow(T, cmap='gray_r')
  plt.title("real_edges")  
  return T;

In [ ]:
res = real_edges(img, 0.2)

###1.3.2

In [ ]:
def evaluation(noisy_image, sigma, th, laplacian, image, th_real):
  T_init = real_edges(image, th_real)
  D_init = EdgeDetect(noisy_image, sigma, th, laplacian)
  (rowsT, columnsT) = T_init.shape
  (rowsD, columnsD) = D_init.shape
  T = set()
  D = set()
  for i in range (rowsT):
    for j in range (columnsT):
      if (T_init[i][j]==1):
        T.add((i,j))
  for i in range (rowsD):
    for j in range (columnsD):
      if (D_init[i][j]==1):
        D.add((i,j))
  intersection = T.intersection(D)
  precision = len(intersection) / len(D)
  recall = len(intersection) / len(T)
  C = (precision + recall) / 2
  print("Precesion = ",precision)
  print("Recall = ",recall)
  print("C = ",C)
  return C;

In [ ]:
res = evaluation(noisy1, 2, 0.00001, 'nonlinear', img, 0.1)

###1.3.3

####Είσοδος με PSNR = 20 dB

In [ ]:
res = EdgeDetect(noisy1, 1.5, 0.2, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1, 0.2, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 2, 0.2, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1.5, 0.3, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1.5, 0.17, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1.5, 0.2, 'nonlinear')

####Είσοδος με PSNR = 10 dB

In [ ]:
res = EdgeDetect(noisy2, 3, 0.2, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1.5, 0.2, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1, 0.2, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1.5, 0.15, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1.5, 0.25, 'linear')

In [ ]:
res = EdgeDetect(noisy1, 1.5, 0.2, 'nonlinear')

##1.4. Εφαρμογή των Αλγορίθμων Ανίχνευσης Ακμών σε Πραγματικές εικόνες

###1.4.1

In [ ]:
img2 = cv2.imread('urban_edges.jpg', cv2.IMREAD_GRAYSCALE)
plt.figure()
img2 = img2.astype(np.float)/255
plt.figure()
plt.imshow(img2, cmap='gray')
plt.title("urban_edges")

In [ ]:
res = real_edges(img2, 0.2)

In [ ]:
def EdgeDetectColor(image, sigma, th):
  Y = zerocrossingsL(image, sigma)
  gradimage = np.gradient(image)
  rows, columns = gradimage[0].shape
  gradarray = np.zeros((rows, columns))
  for i in range (rows):
    for j in range (columns):
      gradarray[i][j] = np.sqrt(gradimage[0][i][j]**2 + gradimage[1][i][j]**2)
  limit = th * np.amax(gradarray)
  (rowsY, columnsY) = Y.shape
  for i in range (rowsY):
    for j in range (columnsY):
      if (Y[i][j] == 1):
        if ((gradarray[i][j]) <= limit):
          Y[i][j] = 0
  D = Y
  plt.figure()
  plt.imshow(D, cmap = 'gray_r')
  plt.title("EdgeDetect")
  return D  

In [ ]:
res = EdgeDetectColor(img2, 1, 0.2)

In [ ]:
def evaluation_color(image, sigma, th, th_real):
  T_init = real_edges(image, th_real)
  D_init = EdgeDetectColor(image, sigma, th)
  (rowsT, columnsT) = T_init.shape
  (rowsD, columnsD) = D_init.shape
  T = set()
  D = set()
  for i in range (rowsT):
    for j in range (columnsT):
      if (T_init[i][j]==1):
        T.add((i,j))
  for i in range (rowsD):
    for j in range (columnsD):
      if (D_init[i][j]==1):
        D.add((i,j))
  intersection = T.intersection(D)
  precision = len(intersection) / len(D)
  recall = len(intersection) / len(T)
  C = (precision + recall) / 2
  print("Precesion = ",precision)
  print("Recall = ",recall)
  print("C = ",C)
  return C;

In [ ]:
res = evaluation_color(img2, 1, 0.15, 0.2)

###1.4.2

In [ ]:
res = EdgeDetectColor(img2, 1, 0.2)

In [ ]:
res = EdgeDetectColor(img2, 2, 0.2)

In [ ]:
res = EdgeDetectColor(img2, 1, 0.05)

#Μέρος 2: Ανίχνευση Σημείων Ενδιαφέροντος (Interest Point Detection)

In [ ]:
def interest_points_visualization(I_, kp_data_, ax=None):
    '''
    Plot keypoints chosen by detectos on image.
    Args:
        I_: Image (if colored, make sure it is in RGB and not BGR).
        kp_data_: Nx3 array, as described in assignment.
        ax: Matplotlib axis to plot on (if None, a new Axes object is created).
    Returns:
        ax: Matplotlib axis where the image was plotted.
    '''
    try:
        I = np.array(I_)
        kp_data = np.array(kp_data_)
    except:
        print('Conversion to numpy arrays failed, check if the inputs (image and keypoints) are in the required format.')
        exit(2)

    try:
        assert(len(I.shape) == 2 or (len(I.shape) == 3 and I.shape[2] == 3))
    except AssertionError as e:
        print('interest_points_visualization: Image must be either a 2D matrix or a 3D matrix with the last dimension having size equal to 3.', file=sys.stderr)
        exit(2)

    try:
        assert(len(kp_data.shape) == 2 and kp_data.shape[1] == 3)
    except AssertionError as e:
        print('interest_points_visualization: kp_data must be a 2D matrix with 3 columns.', file=sys.stderr)
        exit(2)

    if ax is None:
        _, ax = plt.subplots()

    ax.set_aspect('equal')
    ax.imshow(I)
    ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)

    for i in range(len(kp_data)):
        x, y, sigma = kp_data[i]
        circ = Circle((x, y), 3*sigma, edgecolor='g', fill=False, linewidth=2)
        ax.add_patch(circ)

    return ax

In [ ]:
img3 = cv2.imread('mars.png')
plt.figure()
plt.imshow(img3)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(img3)
img3_gray = cv2.cvtColor(img3, cv2.COLOR_RGB2GRAY)
img3_gray = img3_gray.astype(np.float)/255

In [ ]:
img4 = cv2.imread('blood_smear.jpg')
plt.figure()
plt.imshow(img4)
img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(img4)
img4_gray = cv2.cvtColor(img4, cv2.COLOR_RGB2GRAY)
img4_gray = img4_gray.astype(np.float)/255

##2.1. Ανίχνευση Γωνιών

###2.1.1

In [ ]:
def find_J(image, sigma, r):
  n1 = int(2*np.ceil(3*sigma)+1)
  gauss1D = cv2.getGaussianKernel(n1, sigma) # Column vector
  gauss2D = gauss1D @ gauss1D.T # Symmetric gaussian kernel
  Is = cv2.filter2D(image, -1, gauss2D)
  n2 = int(2*np.ceil(3*r)+1)
  gauss1Dr = cv2.getGaussianKernel(n2, r) # Column vector
  gauss2Dr = gauss1Dr @ gauss1Dr.T # Symmetric gaussian kernel
  der_x = cv2.Sobel(Is,cv2.CV_64F,1,0)
  der_y = cv2.Sobel(Is,cv2.CV_64F,0,1)
  mul1 = np.multiply(der_x, der_x)
  mul2 = np.multiply(der_x, der_y)
  mul3 = np.multiply(der_y, der_y)
  J1 = cv2.filter2D(mul1, -1, gauss2Dr)
  plt.figure()
  plt.imshow(J1, cmap = 'gray_r')
  J2 = cv2.filter2D(mul2, -1, gauss2Dr)
  plt.figure()
  plt.imshow(J2, cmap = 'gray_r')
  J3 = cv2.filter2D(mul3, -1, gauss2Dr)
  plt.figure()
  plt.imshow(J3, cmap = 'gray_r')
  return (J1, J2, J3)

In [ ]:
(J1, J2, J3) = find_J(img2, 2, 2.5)

In [ ]:
find_J(img3_gray, 2, 2.5)

In [ ]:
find_J(img4_gray, 2, 2.5)

###2.1.2

In [ ]:
def eigenvalues(image, sigma, r):
  (J1, J2, J3) = find_J(image, sigma, r)
  l_plus = 0.5 * (J1 + J3 + np.sqrt(np.multiply(J1-J3, J1-J3) + 4*np.multiply(J2, J2)))
  plt.figure()
  plt.imshow(l_plus, cmap = 'gray_r')
  l_minus = 0.5 * (J1 + J3 - np.sqrt(np.multiply(J1-J3, J1-J3) + 4*np.multiply(J2, J2)))
  plt.figure()
  plt.imshow(l_minus, cmap = 'gray_r')
  return(l_plus, l_minus)

In [ ]:
(l_plus, l_minus) = eigenvalues(img2, 2, 2.5)

###2.1.3

In [ ]:
def find_R(image, sigma, r, k):
  (l_plus, l_minus) = eigenvalues(image, sigma, r)
  R = np.multiply(l_plus, l_minus) - k * np.multiply(l_plus + l_minus, l_plus + l_minus)
  plt.figure()
  plt.imshow(R, cmap = 'gray_r')
  return R

In [ ]:
def disk_strel(n):
    '''
        Return a structural element, which is a disk of radius n.
    '''
    r = int(np.round(n))
    d = 2*r+1
    x = np.arange(d) - r
    y = np.arange(d) - r
    x, y = np.meshgrid(x,y)
    strel = x**2 + y**2 <= r**2
    return strel.astype(np.uint8)

In [ ]:
def choose_corners(image, sigma, r, k, th_corn):
  R = find_R(image, sigma, r, k)
  ns = np.ceil(3*sigma)*2+1
  B_sq = disk_strel(ns)
  Cond1 = ( R==cv2.dilate(R,B_sq) )
  Cond2 = ( R > th_corn * np.amax(R) )
  Cond_total = (Cond1 & Cond2)
  result = np.where(Cond_total == True)
  listOfCoordinates= list(zip(result[0], result[1]))
  arr = []
  for coordinate in listOfCoordinates:
   arr.append([coordinate[1], coordinate[0], sigma])
  return np.array(arr)

In [ ]:
arr = choose_corners(img2, 1.5, 2.5, 0.05, 0.005)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("urban_edges.jpg"), cv2.COLOR_BGR2RGB), arr)
plt.show()

In [ ]:
arr = choose_corners(img3_gray, 1.5, 2.5, 0.05, 0.005)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("mars.png"), cv2.COLOR_BGR2RGB), arr)
plt.show()

In [ ]:
arr = choose_corners(img4_gray, 1.5, 2.5, 0.05, 0.005)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("blood_smear.jpg"), cv2.COLOR_BGR2RGB), arr)
plt.show()

##2.2. Πολυκλιμακωτή Ανίχνευση Γωνιών


###2.2.1

In [ ]:
def multiscale_choose_corners (image, sigma, r, k, th_corn, N, s):
  sigma_new = sigma * pow(s, N)
  r_new = r * pow(s, N)
  result = choose_corners(image, sigma_new, r_new, k, th_corn)
  return result

In [ ]:
res = multiscale_choose_corners (img2, 2, 2.5, 0.05, 0.005, 1, 1.5)

###2.2.2

In [ ]:
def criterion(image, sigma, r, k, th_corn, N, s):
  sigma_new = sigma * pow(s, N)
  r_new = r * pow(s, N)
  normalized_log = pow(sigma_new, 2) * np.absolute(logfil(image, sigma_new))
  plt.imshow(normalized_log, cmap='gray')
  return normalized_log

In [ ]:
def Gauss(img,sigma):
  n = int(2*np.ceil(3*sigma)+1)
  gauss1D = cv2.getGaussianKernel(n, sigma) # Column vector
  gauss2D = gauss1D @ gauss1D.T # Symmetric gaussian kernel
  gaussian = cv2.filter2D(img, 6, gauss2D)
  return gaussian/gaussian.max()
  

def LoG(img,sigma):
  n = int(2*np.ceil(3*sigma)+1)
  gaussfiltered  = Gauss(img,sigma)
  laplacian = cv2.Laplacian(gaussfiltered, -1, ksize=n)
  return laplacian/laplacian.max()

def LoG_arr(img,sigma0, scale, N):
	abs_log_array = [0]*N
	for i in range(0, N):
		sigma = sigma0*scale**i
		n = int(2*np.ceil(3*sigma)+1)
		gaussfiltered  = Gauss(img,sigma)
		abs_log_array[i] = sigma*sigma*np.abs(cv2.Laplacian(gaussfiltered, cv2.CV_64F ))
	return abs_log_array


In [ ]:
def multiscale_corner_detection(image, sigma0, r, k, theta_corn, N, scale):
	
	if isinstance(image, str):
		gray = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
	else:
		gray = image
	abs_lap_o_gaus = LoG_arr(gray, sigma0, scale, N)
	arr = []

	for i in range(0,N):


		########################2.2.1########################
		sigma = sigma0*scale**i
		ns = (ceil(3*sigma))*2 + 1
		nr = (ceil(3*r))*2 + 1
		Gs = cv2.getGaussianKernel(ns, sigma)
		Gr = cv2.getGaussianKernel(nr, r)
		Is = cv2.filter2D(gray, -1,  Gs)
		[dx,dy] = np.gradient(Is)
		J1 = cv2.filter2D(np.multiply(dx,dx), -1,  Gr)
		J2 = cv2.filter2D(np.multiply(dx,dy), -1,  Gr)
		J3 = cv2.filter2D(np.multiply(dy,dy), -1,  Gr)

		J1_J3squared = (J1-J3)*(J1-J3)
		J2_squared = 4*np.square(J2)
		l_pos = 0.5*(J1+J3+np.sqrt(J1_J3squared+J2_squared))
		l_neg = 0.5*(J1+J3-np.sqrt(J1_J3squared+J2_squared))


		R = l_neg*l_pos - k*(np.square(l_neg+l_pos))
		R_max = np.amax(R)
		B_sq = disk_strel(ns)
		Cond1 = ( R==cv2.dilate(R,B_sq) )
		Cond2 = R > theta_corn*R_max
		if i == 0:
			AbsLogCond = (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i+1]  )
		elif i == N-1:
			AbsLogCond = (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i-1]  )
		else:
			AbsLogCond = np.logical_and( (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i+1]  ), (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i-1]  )  )
		cond = np.logical_and(np.logical_and(Cond1, Cond2), AbsLogCond)
		result = np.where(cond == True)
		listOfCoordinates= list(zip(result[0], result[1]))
	
		for coordinate in listOfCoordinates:
			arr.append([coordinate[1], coordinate[0], sigma])
	return np.array(arr)


In [ ]:
arr = multiscale_corner_detection(img4_gray, 3, 2.5, 0.05, 0.005, 5, 1.5)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("blood_smear.jpg"), cv2.COLOR_BGR2RGB), arr)
plt.show()

In [ ]:
arr = multiscale_corner_detection(img3_gray, 3, 2.5, 0.05, 0.005, 5, 1.5)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("mars.png"), cv2.COLOR_BGR2RGB), arr)
plt.show()

##2.3. Ανίχνευση Blobs

In [ ]:
def blob_detection(image, sigma, threshold):

	########################2.3.1########################
	if isinstance(image, str):
		gray = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
	else:
		gray = image
	ns = (ceil(3*sigma))*2 + 1
	Gs = cv2.getGaussianKernel(ns, sigma)
	Is = cv2.filter2D(gray, -1,  Gs)
	[gx,gy] = np.gradient(Is)
	[hxx,hxy] = np.gradient(gx)
	[hyx,hyy] = np.gradient(gy)
	
	R = hxx*hyy - hxy*hxy

	# cv2.imshow("R", R)
	# cv2.waitKey()

	########################2.3.2########################
	R_max = R.max()
	B_sq = disk_strel(ns)
	Cond1 = (R == cv2.dilate(R,B_sq))
	Cond2 = (R > threshold*R_max)
	cond = Cond1&Cond2
	result = np.where(cond == True)
	listOfCoordinates= list(zip(result[0], result[1]))
	arr = []
	for coordinate in listOfCoordinates:
		arr.append([coordinate[1], coordinate[0], sigma])
	return np.array(arr)

In [ ]:
arr = blob_detection(img3_gray, 1, 0.05)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("mars.png"), cv2.COLOR_BGR2RGB), arr)
plt.show()


In [ ]:
arr = blob_detection(img4_gray, 1, 0.05)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("blood_smear.jpg"), cv2.COLOR_BGR2RGB), arr)
plt.show()

##2.4. Πολυκλιμακωτή Ανίχνευση Blobs

###2.4.1

In [ ]:
def multiscale_blob_detection(image, sigma0, threshold, scale, N):

    ########################2.4.1########################
    if isinstance(image, str):
        gray = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
    else:
        gray = image
    abs_lap_o_gaus = LoG_arr(gray, sigma0, scale, N)
    arr = []


    for i in range(0,N):
        sigma = sigma0*scale**i
        ns = (ceil(3*sigma))*2 + 1
        Gs = cv2.getGaussianKernel(ns, sigma)
        Is = cv2.filter2D(gray, -1,  Gs)
        Gs = cv2.getGaussianKernel(ns, sigma)
        Is = cv2.filter2D(gray, -1,  Gs)
        [gx,gy] = np.gradient(Is)
        [hxx,hxy] = np.gradient(gx)
        [hyx,hyy] = np.gradient(gy)
        
        R = hxx*hyy - hxy*hxy
        
        if i == 0:
            AbsLogCond = (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i+1]  )
        elif i == N-1:
            AbsLogCond = (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i-1]  )
        else:
            AbsLogCond = np.logical_and( (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i+1]  ), (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i-1]  )  )

        R_max = R.max()
        B_sq = disk_strel(ns)
        Cond1 = (R == cv2.dilate(R,B_sq))
        Cond2 = (R >= threshold*R_max)
        cond = Cond1&Cond2&AbsLogCond
        result = np.where(cond == True)
        listOfCoordinates= list(zip(result[0], result[1]))
    
        for coordinate in listOfCoordinates:
            arr.append([coordinate[1], coordinate[0], sigma])
    return np.array(arr)


In [ ]:
arr = multiscale_blob_detection(img3_gray, 2.5, 0.1, 1.5, 5)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("mars.png"), cv2.COLOR_BGR2RGB), arr)
plt.show()


In [ ]:
arr = multiscale_blob_detection(img4_gray, 2.5, 0.1, 1.5, 5)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("blood_smear.jpg"), cv2.COLOR_BGR2RGB), arr)
plt.show()

##2.5. Επιτάχυνση με την χρήση Box Filters και Ολοκληρωτικών Εικόνων (Integral Images)

In [ ]:
def integral_image(image):
  S = np.cumsum(np.cumsum(image, axis=0), axis=1)
  plt.imshow(S)
  return S

In [ ]:
integral_image(img3_gray)

In [ ]:
def compute_integral(integral, shiftX, shiftY, offsetx, offsety):
    shiftX = -int(shiftX)
    shiftY = -int(shiftY)
    offsetx = -int(offsetx)
    offsety = -int(offsety)

    # Right-Bottom corner
    sD = np.roll(integral,-1*shiftY + offsety,axis = 0)
    sD = np.roll(sD,-1*shiftX + offsetx, axis = 1)
    # Left-Top Corner
    sA = np.roll(integral,shiftY + offsety,axis = 0)
    sA = np.roll(sA,shiftX + offsetx,axis = 1)
    # Right-Top corner
    sB = np.roll(integral,shiftY + offsety,axis = 0)
    sB = np.roll(sB,-1*shiftX + offsetx,axis = 1)
    # Left-Bottom Corner
    sC = np.roll(integral,-1*shiftY + offsety,axis = 0)
    sC = np.roll(sC,shiftX + offsetx,axis = 1)
    return [sA,sB,sC,sD]

In [ ]:
def padarray(array, offset):
    return np.pad(array, offset, 'constant', constant_values=0)

In [ ]:
def unpad(array, offset):
    return array[offset:-offset, offset:-offset]

In [ ]:
def boxfilters(image, sigma, threshold, scale, N, multi = False):
    ########################2.5.2########################
    if isinstance(image, str):
        gray = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
    else:
        gray = image    
    n = 2*ceil(3*sigma) + 1

    Ipadded = padarray(gray,floor(n/2))
    integral = integral_image(Ipadded)
 
    xDxx = 2*floor(n/6) + 1
    yDxx = 4*floor(n/6) + 1
    xDyy = 4*floor(n/6) + 1
    yDyy = 2*floor(n/6) + 1
    xDxy = 2*floor(n/6) + 1
    yDxy = 2*floor(n/6) + 1

    ######################## Lxx ########################
    # Central Box
    magnitude = -2
    shiftX = (xDxx -1)/2
    shiftY = (yDxx -1)/2
    pad = floor(n/2) 
    [ tsA,tsB,tsC,tsD ] = compute_integral( integral, shiftX, shiftY, 0, 0 )
    sA = unpad(tsA,pad)
    sB = unpad(tsB,pad)
    sC = unpad(tsC,pad)
    sD = unpad(tsD,pad)

    Lxx = magnitude*(sD + sA - sB - sC) 

    # Left and Right Box
    magnitude = 1
    shiftX = (xDxx -1)/2 + xDxx
    shiftY = (yDxx -1)/2
    pad = floor(n/2) 
    [ tsA,tsB,tsC,tsD ] = compute_integral( integral, shiftX, shiftY, 0, 0 )
    sA = unpad(tsA,pad)
    sB = unpad(tsB,pad)
    sC = unpad(tsC,pad)
    sD = unpad(tsD,pad)

    Lxx = magnitude*(sD + sA - sB - sC) + Lxx

    ######################## Lyy ########################
    # Central Box
    magnitute = -2
    shiftX = (xDyy -1)/2
    shiftY = (yDyy -1)/2
    [ tsA,tsB,tsC,tsD ] = compute_integral( integral, shiftX, shiftY ,0, 0)
    sA = unpad(tsA,pad)
    sB = unpad(tsB,pad)
    sC = unpad(tsC,pad)
    sD = unpad(tsD,pad)

    Lyy = magnitude*(sD + sA - sB - sC)

    # Top and Bottom Box
    magnitude = 1
    shiftX = (xDyy -1)/2
    shiftY = (yDyy -1)/2 + yDyy
    [ tsA,tsB,tsC,tsD ] = compute_integral( integral, shiftX, shiftY , 0 , 0)
    sA = unpad(tsA,pad)
    sB = unpad(tsB,pad)
    sC = unpad(tsC,pad)
    sD = unpad(tsD,pad)

    Lyy = magnitude*(sD + sA - sB - sC) + Lyy




    ######################## Lxy ########################
    # Θεωρούμε πως η ενδιάμεση λωρίδα απο pixels ανάμεσα στα παράθυρα έχει
    # πάχος 1
    if( ceil((n-2*xDxy)/3)%2 == 1):
        rDxy = ceil((n-2*xDxy)/3)
    else:
        rDxy = floor((n-2*xDxy)/3)


    # Top Right Box
    magnitute = -1
    offsetx = -1*(rDxy-1)/2 - (xDxy -1)/2
    offsety = (rDxy-1)/2 + (yDxy -1)/2
    shiftX = (xDxy -1)/2
    shiftY = (yDxy -1)/2
    [ tsA,tsB,tsC,tsD ] = compute_integral( integral, shiftX, shiftY, offsetx, offsety )
    sA = unpad(tsA,pad)
    sB = unpad(tsB,pad)
    sC = unpad(tsC,pad)
    sD = unpad(tsD,pad)

    Lxy = magnitude*(sD + sA - sB - sC)

    # Top Left Box
    magnitute = 1
    offsetx = (rDxy-1)/2 + (xDxy -1)/2
    offsety = (rDxy-1)/2 + (yDxy -1)/2
    shiftX = (xDxy -1)/2
    shiftY = (yDxy -1)/2
    [ tsA,tsB,tsC,tsD ] = compute_integral( integral, shiftX, shiftY, offsetx, offsety  )
    sA = unpad(tsA,pad)
    sB = unpad(tsB,pad)
    sC = unpad(tsC,pad)
    sD = unpad(tsD,pad)

    Lxy = magnitude*(sD + sA - sB - sC) + Lxy

    # Bottom Left Box
    magnitute = -1
    offsetx = (rDxy-1)/2 + (xDxy -1)/2
    offsety = -1*(rDxy-1)/2 - (yDxy -1)/2
    shiftX = (xDxy -1)/2
    shiftY = (yDxy -1)/2
    [ tsA,tsB,tsC,tsD ] = compute_integral( integral, shiftX, shiftY, offsetx, offsety  )
    sA = unpad(tsA,pad)
    sB = unpad(tsB,pad)
    sC = unpad(tsC,pad)
    sD = unpad(tsD,pad)

    Lxy = magnitude*(sD + sA - sB - sC) + Lxy

    # Bottom Right Box
    magnitute = 1
    offsetx = -1*(rDxy-1)/2 - (xDxy -1)/2
    offsety = -1*(rDxy-1)/2 - (yDxy -1)/2
    shiftX = (xDxy -1)/2
    shiftY = (yDxy -1)/2
    [ tsA,tsB,tsC,tsD ] = compute_integral( integral, shiftX, shiftY, offsetx, offsety  )
    sA = unpad(tsA,pad)
    sB = unpad(tsB,pad)
    sC = unpad(tsC,pad)
    sD = unpad(tsD,pad)

    Lxy = magnitude*(sD + sA - sB - sC) + Lxy
    ########################2.5.3########################
    R = Lxx*Lyy - (0.9*Lxy)**2
    R_max = R.max()
    B_sq = disk_strel(n)
    Cond1 = (R == cv2.dilate(R,B_sq))
    Cond2 = (R > threshold*R_max)
    cond = Cond1&Cond2
    result = np.where(cond == True)
    listOfCoordinates= list(zip(result[0], result[1]))
    if multi:
        return cond
    arr = []
    for coordinate in listOfCoordinates:
        arr.append([coordinate[1], coordinate[0], sigma])

    return np.array(arr)

#    print(Lxx)
#   print(Lyy)
#    print(Lxy)
#    plt.figure()
#    plt.imshow(Lxx)
#    plt.title('Lxx')
#    plt.figure()
#    plt.imshow(Lyy)
#    plt.title('Lyy')
#    plt.figure()
#    plt.imshow(Lxy)
#    plt.title('Lxy')

#    return(Lxx, Lyy, Lxy)


In [ ]:
def multiscale_boxfilters(image, sigma0, threshold, scale, N):
	########################2.5.4########################
	if isinstance(image, str):
		gray = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2GRAY).astype(np.float64)/255
	else:
		gray = image
	abs_lap_o_gaus = LoG_arr(gray, sigma0, scale, N)
	arr = []

	for i in range(0,N):
		sigma = sigma0*scale**i
		interestmap = boxfilters(image, sigma, threshold, scale, 1, multi=True)

		if i == 0:
			AbsLogCond = (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i+1]  )
		elif i == N-1:
			AbsLogCond = (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i-1]  )
		else:
			AbsLogCond = np.logical_and( (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i+1]  ), (  abs_lap_o_gaus[i] >= abs_lap_o_gaus[i-1]  )  )
		cond = interestmap&AbsLogCond
		result = np.where(cond == True)
		listOfCoordinates= list(zip(result[0], result[1]))
	
		for coordinate in listOfCoordinates:
			arr.append([coordinate[1], coordinate[0], sigma])
	return np.array(arr)

In [ ]:
arr = multiscale_boxfilters(img4_gray, 2.5, 0.01, 1.5, 5)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("blood_smear.jpg"), cv2.COLOR_BGR2RGB), arr)
plt.show()

In [ ]:
arr = multiscale_boxfilters(img3_gray, 2.5, 0.01, 1.5, 5)
ax = interest_points_visualization(cv2.cvtColor(cv2.imread("mars.png"), cv2.COLOR_BGR2RGB), arr)
plt.show()

#Μέρος 3: Εφαρμογές σε Ταίριασμα και Κατηγοριοποίηση Εικόνων με Χρήση Τοπικών Περιγραφητών στα Σημεία Ενδιαφέροντος

In [ ]:
import cv21_lab1_part3_utils as p3
#import cv21_lab1_part2_utils as p2

#from cv21_lab1_part3_utils import featuresSURF
#from cv21_lab1_part3_utils import featuresHOG
from cv21_lab1_part3_utils import matching_evaluation
#from cv21_lab1_part3_utils import createTrainTest
#from cv21_lab1_part3_utils import BagOfWords
#from cv21_lab1_part3_utils import svm
#from cv21_lab1_part3_utils import FeatureExtraction

In [ ]:
p3.matching_evaluation?

##3.1. Ταίριασμα Εικόνων υπό Περιστροφή και Αλλαγή Κλίμακας

In [ ]:
!unzip cv21_lab1_part3_material.zip

In [ ]:
!mv glove.* cv21_lab1_part3_material -f

In [ ]:
detect_fun = lambda I: choose_corners(I, 3, 2.5, 0.05, 0.005)

print("########## SURF ##########")
desc_fun = lambda I, kp: p3.featuresSURF(I, kp)
avg_scale_errors, avg_theta_errors = matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

    
print("########## HOG ##########")
desc_fun = lambda I, kp: p3.featuresHOG(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

In [ ]:
detect_fun = lambda I: multiscale_corner_detection(I, 3, 2.5, 0.05, 0.005,4,1.5)

print("########## SURF ##########")
desc_fun = lambda I, kp: p3.featuresSURF(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

    
print("########## HOG ##########")
desc_fun = lambda I, kp: p3.featuresHOG(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

In [ ]:
detect_fun = lambda I: choose_blobs(I, 3, 0.12)

print("########## SURF ##########")
desc_fun = lambda I, kp: p3.featuresSURF(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

    
print("########## HOG ##########")
desc_fun = lambda I, kp: p3.featuresHOG(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

In [ ]:
detect_fun = lambda I: merged_choice_blobs(I, 3, 0.12, 6, 1.5)

print("########## SURF ##########")
desc_fun = lambda I, kp: p3.featuresSURF(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

    
print("########## HOG ##########")
desc_fun = lambda I, kp: p3.featuresHOG(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

In [ ]:
detect_fun = lambda I: merged_choice_points_of_interest(I, 2.5, 0.05, 4, 1.5)

print("########## SURF ##########")
desc_fun = lambda I, kp: p3.featuresSURF(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i]))

    
print("########## HOG ##########")
desc_fun = lambda I, kp: p3.featuresHOG(I, kp)
avg_scale_errors, avg_theta_errors = p3.matching_evaluation(detect_fun, desc_fun)
print(avg_scale_errors)
print(avg_theta_errors)

for i in range(3):
    print('Avg. Scale Error for Image {:.3f}: {:.3f}'.format(i,avg_scale_errors[i]))
    print('Avg. Theta Error for Image {:.3f}: {:.3f}'.format(i,avg_theta_errors[i])) 

##3.2. Κατηγοριοποίηση Εικόνων

###3.2.1, 3.2.2

In [ ]:
detect_fun = lambda I: merged_choice(I, 3, 2.5, 0.05, 0.005,4,1.5)

print("########## SURF ##########")
desc_fun = lambda I, kp: p3.featuresSURF(I, kp)
val1 = p3.FeatureExtraction(detect_fun, desc_fun)
    
print("########## HOG ##########")
desc_fun = lambda I, kp: p3.featuresHOG(I, kp)
val2 = p3.FeatureExtraction(detect_fun, desc_fun)

In [ ]:
accs = []
for k in range(5):
    # Split into a training set and a test set.
    data_train, label_train, data_test, label_test = p3.createTrainTest(val1, k)

    # Perform Kmeans to find centroids for clusters.
    BOF_tr, BOF_ts = p3.BagOfWords(data_train, data_test)

    # Train an svm on the training set and make predictions on the test set
    acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
    accs.append(acc)
print(accs)
print('Mean accuracy for Harris-Laplace with SURF descriptors: {:.3f}%'.format(100.0*np.mean(accs)))
accs = []
for k in range(5):
    # Split into a training set and a test set.
    data_train, label_train, data_test, label_test = p3.createTrainTest(val2, k)

    # Perform Kmeans to find centroids for clusters.
    BOF_tr, BOF_ts = p3.BagOfWords(data_train, data_test)

    # Train an svm on the training set and make predictions on the test set
    acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
    accs.append(acc)

print('Mean accuracy for Harris-Laplace with HOG descriptors: {:.3f}%'.format(100.0*np.mean(accs)))
print(accs)


In [ ]:
detect_fun = lambda I:  merged_choice_blobs(I, 3, 0.12, 6, 1.5)

print("########## SURF ##########")
desc_fun = lambda I, kp: p3.featuresSURF(I, kp)
val3 = p3.FeatureExtraction(detect_fun, desc_fun)
    
print("########## HOG ##########")
desc_fun = lambda I, kp: p3.featuresHOG(I, kp)
val4 = p3.FeatureExtraction(detect_fun, desc_fun)

In [ ]:
accs = []
for k in range(5):
    # Split into a training set and a test set.
    data_train, label_train, data_test, label_test = p3.createTrainTest(val3, k)

    # Perform Kmeans to find centroids for clusters.
    BOF_tr, BOF_ts = p3.BagOfWords(data_train, data_test)

    # Train an svm on the training set and make predictions on the test set
    acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
    accs.append(acc)
print(accs)

print('Mean accuracy for Blob Detection with SURF descriptors: {:.3f}%'.format(100.0*np.mean(accs)))
accs = []
for k in range(5):
    # Split into a training set and a test set.
    data_train, label_train, data_test, label_test = p3.createTrainTest(val4, k)

    # Perform Kmeans to find centroids for clusters.
    BOF_tr, BOF_ts = p3.BagOfWords(data_train, data_test)

    # Train an svm on the training set and make predictions on the test set
    acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
    accs.append(acc)

print('Mean accuracy for Blob Detection with HOG descriptors: {:.3f}%'.format(100.0*np.mean(accs)))
print(accs)

In [ ]:
detect_fun = lambda I: merged_choice_points_of_interest(I, 2.5, 0.05, 4, 1.5)

print("########## SURF ##########")
desc_fun = lambda I, kp: p3.featuresSURF(I, kp)
val5 = p3.FeatureExtraction(detect_fun, desc_fun)
    
print("########## HOG ##########")
desc_fun = lambda I, kp: p3.featuresHOG(I, kp)
val6 = p3.FeatureExtraction(detect_fun, desc_fun)

In [ ]:
accs = []
for k in range(5):
    # Split into a training set and a test set.
    data_train, label_train, data_test, label_test = p3.createTrainTest(val5, k)

    # Perform Kmeans to find centroids for clusters.
    BOF_tr, BOF_ts = p3.BagOfWords(data_train, data_test)

    # Train an svm on the training set and make predictions on the test set
    acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
    accs.append(acc)
print(accs)

print('Mean accuracy for Box Filters with SURF descriptors: {:.3f}%'.format(100.0*np.mean(accs)))


accs = []
for k in range(5):
    # Split into a training set and a test set.
    data_train, label_train, data_test, label_test = p3.createTrainTest(val6, k)

    # Perform Kmeans to find centroids for clusters.
    BOF_tr, BOF_ts = p3.BagOfWords(data_train, data_test)

    # Train an svm on the training set and make predictions on the test set
    acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
    accs.append(acc)

print('Mean accuracy for Box Filters with HOG descriptors: {:.3f}%'.format(100.0*np.mean(accs)))
print(accs)


###3.2.3, 3.2.4

In [ ]:
def BoVW_hist(features, centers, kmeans):
    distances = np.zeros((features.shape[0], centers.shape[0]))
    for  descriptor in range(0, features.shape[0]):
        for center in range(0, centers.shape[0]):
            sum = 0
            for i in range(0, features[descriptor].shape[0]):
                 sum += (features[descriptor][i] - centers[center][i])**2
            distances[descriptor][center] = sqrt(sum)
    mins = np.min(distances, axis = 1)   
#     Distances = kmeans.transform(features)
#     #print(Distances.shape)
#     #minx, miny = list(zip( np.where(Distance == np.amin(Distance))[0],np.where(Distance == np.amin(Distance))[1] ))[0]
#     mins = np.min(Distances, axis=1) 
    hist, bin_edges = np.histogram(mins, bins=500, density=True)
    #normalized = hist/norm(hist)
    #return normalized, bin_edges
    return hist, bin_edges

In [ ]:
def BoVW(data_train, data_test, test = False):
    #number of clusters that are going to be created with kmeans clustering:
    cluster_number =  [500,750,1000,1250,1500,1750,2000]
    #percentage of initial descriptors to be implemented upon kmeans:
    percentage = 0.5
    #concatenation of training data:
    vectors = np.concatenate(data_train)

    N = vectors.shape[0]
    #random subset of the initial vectors:
    subsetOfvectors = []
    for i in range(0, floor(percentage*N)):
        temp = random.randint(0,N-1) 
        subsetOfvectors.append(vectors[temp])


    if test:    
        wcss = []
        for i in cluster_number:
            kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
            kmeans.fit(subsetOfvectors)
            wcss.append(kmeans.inertia_)
        plt.plot(cluster_number, wcss)
        plt.title('Elbow Method')
        plt.xlabel('Number of clusters')
        plt.ylabel('WCSS')
        plt.show()
        return 
    
    
    kmeans = KMeans(n_clusters=500, init='k-means++', max_iter=50, n_init=10, random_state=0)
    kmeans.fit(subsetOfvectors)
    centers = kmeans.cluster_centers_
#     plt.scatter(centers[:, 0], centers[:, 1])
#     plt.show()

   
    BOF_tr = []
    BOF_ts = []
    for i in range(0,len(data_train)):
        hist, bin_edges = BoVW_hist(data_train[i], centers, kmeans)
#         if i<5:
#             print(hist.shape)
#             plt.hist(hist, bins=bin_edges)
#             plt.show()
        BOF_tr.append(hist)
       
    for i in range(0,len(data_test)):
        hist, bin_edges = BoVW_hist(data_test[i], centers, kmeans)
        BOF_ts.append(hist)
        
    return BOF_tr, BOF_ts

In [ ]:
accs = []
"""
for k in range(5):
    data_train, label_train, data_test, label_test = p3.createTrainTest(val1, k)
    # Perform Kmeans to find centroids for clusters.
    BOF_tr, BOF_ts = BoVW(data_train, data_test)
    #centers = BoVW(data_train, data_test)
    # Train an svm on the training set and make predictions on the test set
    acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
    print(acc)
    accs.append(acc)
print(accs)
"""
data_train, label_train, data_test, label_test = p3.createTrainTest(val1, k)
# Perform Kmeans to find centroids for clusters.
BOF_tr, BOF_ts = BoVW(data_train, data_test)

# Train an svm on the training set and make predictions on the test set
acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
print(acc)
accs.append(acc)
print('Mean accuracy for Harris-Laplace with SURF descriptors: {:.3f}%'.format(100.0*np.mean(accs)))